**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Apr  3 09:20:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 2010_Compiled_MYE_H2_P2_Ans.pdf
 2010_JC1_H2_Chemistry_MYE_P1_answers.pdf
 2010_JC1_H2_Phy_MYE_Sol.pdf
 2011_JC1_H2_Chemistry_MYE_P1_solution_31_3_2011.pdf
'2011_JC1_H2_Phy_MYE (1).pdf'
'2011_JC1_H2_Phy_MYE_Sol (1).pdf'
 2011_MYE_H2_Paper_2_suggested_solutions_for_students.pdf
 2012_H2_chem_mye_p2_20120524_MS.pdf
'2013_H2_MYE_revision_answers (1).pdf'
 2013_H2_MYE_revision_answers.pdf
 2013_JC1_H1_GEO_Litho_Term_2_Essay_Test_1_Mark_Scheme.pdf
'2013_MYE_Mock_Paper_4_(sol).pdf'
'2013_MYE_Mock_Paper_5_(sol).pdf'
'2013_MYE_Mock_Paper_6_(Qns).pdf'
'2013_MYE_Mock_Paper_6_(sol).pdf'
'2013_Term_3_JC1_Timetable_[wef_22_Jul]_(Updated_on_19_Jul).pdf'
'2014_JC2_T3W10_&_Sep_Break_Lessons_22_Aug_updated.xlsx'
 20190128_133735.jpg
 215895823-The-Blue-Planet.pdf
 477.pdf
 5_6086910458860667057.ipynb.gdoc


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
ls

In [6]:
!sed -i 's/batch=1/batch=128/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=32/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 50000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=5@' cfg/yolov3_training.cfg #changed to 2 classes
!sed -i '696 s@classes=80@classes=5@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=5@' cfg/yolov3_training.cfg #End changes
!sed -i '603 s@filters=255@filters=30@' cfg/yolov3_training.cfg # (classes+5)*3
!sed -i '689 s@filters=255@filters=30@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=30@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
ls


3rdparty/               darknet_images.py       net_cam_v3.sh*
backup/                 darknet.py              net_cam_v4.sh*
build/                  darknet_video.py        obj/
build.ps1*              data/                   README.md
build.sh*               image_yolov3.sh*        results/
cfg/                    image_yolov4.sh*        scripts/
cmake/                  include/                src/
CMakeLists.txt          json_mjpeg_streams.sh*  vcpkg.json
darknet*                LICENSE                 video_yolov3.sh*
DarknetConfig.cmake.in  Makefile                video_yolov4.sh*


In [8]:
!echo "Cold sore" > data/obj.names
!echo "Eczema" >> data/obj.names
!echo "Hives" >> data/obj.names
!echo "Vitiligo" >> data/obj.names
!echo "Acne" >> data/obj.names
!echo -e 'classes= 5\ntrain  = data/train.txt\nvalid  = data/valid.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
pwd

'/content/darknet'

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-03 09:21:16--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  23.3MB/s    in 7.3s    

2021-04-03 09:21:24 (21.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/CombinedCEHVA.zip -d data/obj

Archive:  /mydrive/yolov3/CombinedCEHVA.zip
  inflating: data/obj/1_16a0814ebbb.jpg  
  inflating: data/obj/1_16a0814ebbb.txt  
  inflating: data/obj/1_16a0814ebbb_jpg.rf.c5215918793068417024ed5c4f7e860e.jpg  
  inflating: data/obj/1_16a0814ebbb_jpg.rf.c5215918793068417024ed5c4f7e860e.txt  
  inflating: data/obj/1_16a0814ebbb_jpg.rf.ea267ccd208b2c5e8a37f49e1d361ea2.jpg  
  inflating: data/obj/1_16a0814ebbb_jpg.rf.ea267ccd208b2c5e8a37f49e1d361ea2.txt  
  inflating: data/obj/1_jpg.rf.0adecd995b6aa6faeec88f89591648f9.jpg  
  inflating: data/obj/1_jpg.rf.0adecd995b6aa6faeec88f89591648f9.txt  
  inflating: data/obj/1_jpg.rf.10bfbbe78b9c2944912f794f448dfb77.jpg  
  inflating: data/obj/1_jpg.rf.10bfbbe78b9c2944912f794f448dfb77.txt  
  inflating: data/obj/1_jpg.rf.166623df4c4acba921469ad69c42c527.jpg  
  inflating: data/obj/1_jpg.rf.166623df4c4acba921469ad69c42c527.txt  
  inflating: data/obj/1_jpg.rf.4080c2302278362425ad742b6ebfb8da.jpg  
  inflating: data/obj/1_jpg.rf.4080c2302278362425ad742

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4]) # changed here to take in the class index at number[1]
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
['1', '0.6851712843103379', '0.1861025446386146', '0.03927602185762122', '0.048553948597618544']
637 data/obj/woman-with-hard-pimples-on-her-face_jpg.rf.c2d7649e6fd14b11ef0a46cd91c65dd5.txt
1 0.6851712843103379 0.1861025446386146 0.03927602185762122 0.048553948597618544
3 0.5295138888888888 0.4763779527559055 0.4548611111111111 0.4251968503937008

['3', '0.5295138888888888', '0.4763779527559055', '0.4548611111111111', '0.4251968503937008']
638 data/obj/Vitiligo.txt
3 0.5295138888888888 0.4763779527559055 0.4548611111111111 0.4251968503937008
2 0.33250909103232174 0.4858496392049677 0.40671557335408653 0.504256230417308

['2', '0.33250909103232174', '0.4858496392049677', '0.40671557335408653', '0.504256230417308']
639 data/obj/10_jpg.rf.9033abadadc37bf49557d84f5a64bb8e.txt
2 0.33250909103232174 0.4858496392049677 0.40671557335408653 0.504256230417308
2 0.11601792420536713 0.6326001746263353 0.1641941605443638 0.16234655541217954
['2', '

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/herpes-simplex-labialis-21__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd_jpg.rf.4360a82421df451905ccf29eecf242d1.jpg', 'data/obj/Vitiligo-4.jpg', 'data/obj/GettyImages-181707522_jpg.rf.2b614907ee7f623af35d4ea573ed34d5.jpg', 'data/obj/GettyImages-552992761_thumb-732x549.jpg', 'data/obj/54_jpg.rf.7f5ea9ff5afe31e7d56df48fdb442759.jpg', 'data/obj/mistake-for-hives-1_jpg.rf.a759d7b4622763f028bc7feb30f24b8c.jpg', 'data/obj/Can-Alcohol-Cause-Rashes_jpg.rf.f12bcf2449dcee18ea5e1f162b6eb9dd.jpg', 'data/obj/young-woman-has-skin-rash-itch-on-neck-royalty-free-image-1587153399_jpg.rf.0ca4db1b88054b9274e844e45559665c.jpg', 'data/obj/Pimple-vs-boil-shutterstock-2-1_jpg.rf.f568cde31370fc78461e35dc5e8c8b8c.jpg', 'data/obj/male-hand-with-vitiligo-skin-condition-characterized-by-white-unpigmented-patches-or-blotches-MWXN8W_jpg.rf.c3103cf11557bac4ca4c20f6348915e7.jpg', 'data/obj/herpes-simplex-labialis-22__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd_jpg.rf.319fd2dbf5e14dae2ee14

In [13]:
split = 15 #percent
Validation_list = []

print(len(images_list))
validationAmount = round(len(images_list)/15 - 1)
print(validationAmount)

import random
ValiIndex = random.sample(range(len(images_list)), validationAmount)

for i in ValiIndex:
  print(i+1)
  if(len(images_list) > i):
    Validation_list.append(images_list.pop(i))
  else:
    Validation_list.append(images_list.pop(len(images_list)-1))
  
print(Validation_list)


845
55
383
382
753
706
254
497
701
734
2
204
121
129
617
409
816
538
33
738
311
442
197
651
597
646
54
267
794
168
681
180
524
740
276
668
85
198
758
624
18
803
676
81
704
132
735
244
532
743
648
712
514
82
650
510
78
['data/obj/images--4-_jpg.rf.e738941ee01d4f6cd705fb032207b255.jpg', 'data/obj/14_jpg.rf.8342a6db21db31e912993e5b8f2c954e.jpg', 'data/obj/34_jpg.rf.e962630125d3654203c4cd5d223b46a4.jpg', 'data/obj/herpes-simplex-labialis-03__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd_jpg.rf.9a36008f50d9670eba5b5fc00c183795.jpg', 'data/obj/Facial_jpg.rf.8983ba8d124633c3143eb233c1548f45.jpg', 'data/obj/acne-face-13-a20f2898cdab4b579e9f7cb8fc9062eb_jpg.rf.8fb14d18a9b868dc20645dcde5a240ef.jpg', 'data/obj/36_jpg.rf.cdbed1eaef8b44192383aaee6fa02b34.jpg', 'data/obj/How-To-Get-Rid-Of-Pimple-Scars-On-Face.jpg', 'data/obj/Vitiligo-4.jpg', 'data/obj/9_jpg.rf.653b9522e1afaa0e25c7662d1f774387.jpg', 'data/obj/49_jpg.rf.1b53a8b4f715a5997260fe1553075103.jpg', 'data/obj/vitiligo-159x300_jpg.rf.b3

In [14]:
print(len(Validation_list))

55


In [15]:
# images_copy = images_list
# print(len(images_copy))
# print(images_copy.pop(1))
# print(len(images_copy))

# import random
# nigs = random.sample(range(100), 10)
# print(nigs)
# print(nigs[2])
# print(nigs[3])

# for x in nigs:
#   print(x)


In [16]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 
#Create Vali.txt file
valid = open("data/valid.txt", "w")
valid.write("\n".join(Validation_list))
valid.close()

In [17]:
cat data/valid.txt

data/obj/images--4-_jpg.rf.e738941ee01d4f6cd705fb032207b255.jpg
data/obj/14_jpg.rf.8342a6db21db31e912993e5b8f2c954e.jpg
data/obj/34_jpg.rf.e962630125d3654203c4cd5d223b46a4.jpg
data/obj/herpes-simplex-labialis-03__ProtectWyJQcm90ZWN0Il0_FocusFillWzI5NCwyMjIsIngiLDFd_jpg.rf.9a36008f50d9670eba5b5fc00c183795.jpg
data/obj/Facial_jpg.rf.8983ba8d124633c3143eb233c1548f45.jpg
data/obj/acne-face-13-a20f2898cdab4b579e9f7cb8fc9062eb_jpg.rf.8fb14d18a9b868dc20645dcde5a240ef.jpg
data/obj/36_jpg.rf.cdbed1eaef8b44192383aaee6fa02b34.jpg
data/obj/How-To-Get-Rid-Of-Pimple-Scars-On-Face.jpg
data/obj/Vitiligo-4.jpg
data/obj/9_jpg.rf.653b9522e1afaa0e25c7662d1f774387.jpg
data/obj/49_jpg.rf.1b53a8b4f715a5997260fe1553075103.jpg
data/obj/vitiligo-159x300_jpg.rf.b36449c4a2e1af122a0e233ae09b072e.jpg
data/obj/300px-Cold_urticaria3_jpg.rf.1b5f0117e1ca3087cc4d3ef181a5f963.jpg
data/obj/29_jpg.rf.fb7e1ffdc68d0a193952cf8ded069a97.jpg
data/obj/images1_jpg.rf.ebc8cd62161f5bde3eb035b29b1e7d72.jpg
data/obj/herpes-simplex-la

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/ext9_1_yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.835199), count: 6, class_loss = 0.001041, iou_loss = 0.120243, total_loss = 0.121283 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.742144), count: 11, class_loss = 0.530181, iou_loss = 0.408386, total_loss = 0.938567 
 total_bbox = 6631331, rewritten_bbox = 0.199930 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.897381), count: 6, class_loss = 0.000516, iou_loss = 0.049729, total_loss = 0.050245 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.910774), count: 1, class_loss = 0.000089, iou_loss = 0.004199, total_loss = 0.004288 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.803000), count: 4, class_loss = 0.038653, iou_loss = 0.061832, total_loss = 0.100486 
 total_bbox = 6631342, rewritten_bbox = 0.199929 % 
v3

'/content'